In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient


# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
# import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load Motor neuron table

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')
pre_to_mn_df.shape
pre_to_mn_df.sum(axis=0).sum()

In [ ]:
pre_to_mn_df.columns

In [ ]:
All = slice(None)

pre_to_mn_df.loc[:,('L','Accessory','coxa','flex',All,'AcPst')].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.loc[:,('L','Leg','coxa','extend')].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.loc[:,('L','Leg','coxa','extend')].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.loc[:,('L','Leg','coxa','flex')].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.loc[:,('L','Leg','trochanter',All)].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.loc[:,('L','Leg','femur','extend')].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.loc[:,('L','Leg','femur','flex','main_tibia_flexor')].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.loc[:,('L','Leg','femur','flex','auxiliary_tibia_flexor','Bslow')].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.loc[:,('L','Leg','femur','flex','auxiliary_tibia_flexor',['A1','A2','E'])].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.loc[:,('L','Leg','ltm')].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.loc[:,('L','Leg','tibia')].columns.get_level_values('segID').to_list()

In [ ]:
pre_to_mn_df.groupby(level=[0]).size()


In [ ]:
# mn_inputs_df = client.materialize.synapse_query(post_ids = [648518346503918688],timestamp=connectome_create.get_timestamp())
# mn_inputs_df.head()
# mn_inputs_df.shape

In [ ]:
pre_to_mn_df


In [ ]:

mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor_med_venU',
    'tarsus_depressor_noid',
    ]
pre_to_mn_df.head()

new_idx = utils.sort_segment_fcn_index(pre_to_mn_df.columns)
pre_to_mn_df = pre_to_mn_df[new_idx]

pre_to_mn_df.head()

In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

In [ ]:
All = slice(None)
pre_to_mn_df.loc[('local','tibia_extensor'),:].index.get_level_values('segID').to_list()

# Cosine similarity examples in paper

In [ ]:
a = np.array([[0.0291 ,0.0437 ,0.0656] , [0.0423 ,0.0635 ,0.0953], [0.0635 ,0.0953  ,0.1429]])
a = np.round(a/np.min(a)*10)
a[0,0] = a[0,0] + 1
a[0,1] = a[0,1] + 2 
a[0,2] = a[0,2] - 10
print(a)
sim_mat = cosine_similarity(a)
sim_mat

In [ ]:
a = np.array([[0.0291 ,0.0437 ,0.0656] , [0.0423 ,0.0635 ,0.0953], [0.0635 ,0.0953  ,0.1429]])
a = np.round(a/np.min(a)*10)
temp = a[0,0]
a[0,0] = a[2,2]
a[2,2] = temp
print(a)
sim_mat = cosine_similarity(a)
sim_mat

In [ ]:
a = np.array([[30., 5., 5.] , [5., 30., 30.], [5.,30.,5.]])
# a = np.round(a/np.min(a)*10)
print(a)
sim_mat = cosine_similarity(a)
sim_mat

# Cosine similarity matrices

## All

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_all_grayscale.svg',format='svg')

## Local

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['local',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_local_grayscale.svg',format='svg')

## Intersegmental

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['intersegmental',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_intersegmental_grayscale.svg',format='svg')

## Descending

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['descending',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_descending_grayscale.svg',format='svg')

## Ascending

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['ascending',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_ascending_grayscale.svg',format='svg')

## Sensory

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['sensory',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_sensory_grayscale.svg',format='svg')

### Dig into the sensory similarity

In [ ]:
pre_to_mn_df

In [ ]:
pre_to_mn_df.loc[('sensory','ltm'),mpool_dict['ltm']]

In [ ]:
pre_to_mn_df.loc[:,mpool_dict['auxiliary_tibia_flexor_E_wtarsus']].columns.get_level_values('segID').to_list()

# mpool_dict = utils.get_motor_pool_tuple_dict()
# pool_keys = [
#     'thorax_swing',
#     'thorax_stance',
#     'trochanter_extension',
#     'trochanter_flexion',
#     'femur_reductor',
#     'tibia_extensor',
#     'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
#     'auxiliary_tibia_flexor_A',
#     'auxiliary_tibia_flexor_B',
#     'auxiliary_tibia_flexor_E',
#     'ltm',
#     'tarsus_depressor_med_venU',
#     'tarsus_depressor_noid',
#     ]

## Regress local vs. normal, vs. descending, etc.

In [ ]:
mn_mi_df = pre_to_mn_df.columns.to_frame()
mn_mi_df

# Ha! for this, put the tarsus neurons in the right pools
muscle_tuple_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',
    # 'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor',
    ]


    #  'main_tibia_flexor_wtarsus':                 
    #  'auxiliary_tibia_flexor_B_wtarsus':
    #  'auxiliary_tibia_flexor_E_wtarsus':
    #             'tarsus_depressor_medial':
    #             'tarsus_depressor_ventralU':  
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor_wtarsus',
    # 'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B_wtarsus',
    'auxiliary_tibia_flexor_E_wtarsus',
    'ltm',
    'tarsus_depressor_medial',
    'tarsus_depressor_ventralU'
    ]


for key in pool_keys:
    mn_tup = muscle_tuple_dict[key]
    mn_mi_df.loc[mn_tup,'preferred_pool'] = key

mn_mi_df.preferred_pool = mn_mi_df.preferred_pool.astype("category")
mn_mi_df.preferred_pool = mn_mi_df.preferred_pool.cat.set_categories(pool_keys)
mn_mi_df

In [ ]:
sim_mat_all = cosine_similarity(pre_to_mn_df.to_numpy().transpose())
sim_mat_local = cosine_similarity(pre_to_mn_df.loc['local',:].to_numpy().transpose())
sim_mat_desc = cosine_similarity(pre_to_mn_df.loc['descending',:].to_numpy().transpose())
sim_mat_inter = cosine_similarity(pre_to_mn_df.loc['intersegmental',:].to_numpy().transpose())
sim_mat_ascending = cosine_similarity(pre_to_mn_df.loc['ascending',:].to_numpy().transpose())
sim_mat_sensory = cosine_similarity(pre_to_mn_df.loc['sensory',:].to_numpy().transpose())

tril_mask = np.full((sim_mat_all.shape[0], sim_mat_all.shape[1]), True, dtype=bool)
motor_pool_mask = np.full((sim_mat_all.shape[0], sim_mat_all.shape[1]), False, dtype=bool)
i_index = np.full((sim_mat_all.shape[0], sim_mat_all.shape[1]), 0, dtype=int)
j_index = np.full((sim_mat_all.shape[0], sim_mat_all.shape[1]), 0, dtype=int)
for i in range(motor_pool_mask.shape[0]):
    for j in range(motor_pool_mask.shape[1]):
        motor_pool_mask[i,j] =(mn_mi_df.iloc[i,:].preferred_pool==mn_mi_df.iloc[j,:].preferred_pool)
        i_index[i,j] = i
        j_index[i,j] = j

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(motor_pool_mask, xticklabels=mn_mi_df.preferred_pool.to_list(),cmap=plt.get_cmap('Greys'))

# fig = plt.figure(2, figsize = [6,6])
# ax = sns.heatmap(np.tril(motor_pool_mask,-1), xticklabels=mn_mi_df.preferred_pool.to_list(),cmap=plt.get_cmap('Greys'))

tril_mask = np.tril(tril_mask,-1)

pool_arr = mn_mi_df.preferred_pool.to_numpy()
pool_arr = np.tile(pool_arr.reshape((-1,1)),len(pool_arr))
pool_arr.reshape(-1)

In [ ]:
sim_dict = {
    'all_comps':sim_mat_all.reshape(-1),
    'local':sim_mat_local.reshape(-1),
    'desc':sim_mat_desc.reshape(-1),
    'inter':sim_mat_inter.reshape(-1),
    'asce':sim_mat_ascending.reshape(-1),
    'sense':sim_mat_sensory.reshape(-1),
    'pool':pool_arr.reshape(-1),
    'same_pool': motor_pool_mask.reshape(-1),
    'lower_tri':tril_mask.reshape(-1),
    'i_index':i_index.reshape(-1),
    'j_index':j_index.reshape(-1)
    }


In [ ]:

sim_compare_df = pd.DataFrame.from_dict(data = sim_dict,orient='columns')
sim_compare_df
# # get rid of upper triangle
sim_compare_df = sim_compare_df.loc[sim_compare_df.lower_tri.to_numpy()]
sim_compare_df


In [ ]:
cat_pal = {
    True: '#000000',
    False: '#DFDFDF',
    }

pool_pal = {
    'thorax_swing': '#A502AA',
    'thorax_stance': '#00A2B4',
    'trochanter_extension': '#D5CB6C',
    'trochanter_flexion': '#3F42A2',
    'femur_reductor': '#FF0000',
    'tibia_extensor': '#CC8544',
    'main_tibia_flexor_wtarsus': '#2E3191',
    'auxiliary_tibia_flexor_B_wtarsus': '#2DB515',
    'auxiliary_tibia_flexor_E_wtarsus': '#156005',
    'ltm': '#FFF100',
    'tarsus_depressor_medial': '#ff81c0',
    'tarsus_depressor_ventralU': '#AA81c0',
}
    #  'main_tibia_flexor_wtarsus':                 
    #  'auxiliary_tibia_flexor_B_wtarsus':
    #  'auxiliary_tibia_flexor_E_wtarsus':
    #             'tarsus_depressor_medial':
    #             'tarsus_depressor_ventralU':  

fig, ax = plt.subplots( 1, 1, figsize=(12,12))
diff_comps = sim_compare_df.loc[~sim_compare_df.same_pool]
sns.scatterplot(data=diff_comps, x="all_comps", y="local", hue="same_pool",palette = cat_pal,ax=ax,linewidth=0)

same_comps = sim_compare_df.loc[sim_compare_df.same_pool]
sns.scatterplot(data=same_comps, x="all_comps", y="local", hue="same_pool",palette = cat_pal,ax=ax,linewidth=0)

# low_comps = sim_compare_df.loc[sim_compare_df.local<.23].loc[sim_compare_df.same_pool].loc[sim_compare_df.pool.ne('trochanter_flexion')]
# sns.scatterplot(data=low_comps, x="all_comps", y="local", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

# hi_comps = sim_compare_df.loc[sim_compare_df.local>.3].loc[~sim_compare_df.same_pool]
# sns.scatterplot(data=hi_comps, x="all_comps", y="local", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

# down_comps = sim_compare_df.loc[sim_compare_df.local>.4].loc[sim_compare_df.local<sim_compare_df.all_comps]
# sns.scatterplot(data=down_comps, x="all_comps", y="local", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

ax.plot([0, 1],[0,1],color='#000000')

fig.savefig('./figpanels/cosine_sim_local_v_all.svg',format='svg')

In [ ]:
cat_pal = {
    True: '#000000',
    False: '#DFDFDF',
    }

pool_pal = {
    'thorax_swing': '#A502AA',
    'thorax_stance': '#00A2B4',
    'trochanter_extension': '#D5CB6C',
    'trochanter_flexion': '#3F42A2',
    'femur_reductor': '#FF0000',
    'tibia_extensor': '#CC8544',
    'main_tibia_flexor_wtarsus': '#2E3191',
    'auxiliary_tibia_flexor_B_wtarsus': '#2DB515',
    'auxiliary_tibia_flexor_E_wtarsus': '#156005',
    'ltm': '#FFF100',
    'tarsus_depressor_medial': '#ff81c0',
    'tarsus_depressor_ventralU': '#AA81c0',
}

fig, ax = plt.subplots( 1, 1, figsize=(12,12))
diff_comps = sim_compare_df.loc[~sim_compare_df.same_pool]
sns.scatterplot(data=diff_comps, x="all_comps", y="local", hue="same_pool",palette = cat_pal,ax=ax,linewidth=0)

same_comps = sim_compare_df.loc[sim_compare_df.same_pool]
sns.scatterplot(data=same_comps, x="all_comps", y="local", hue="same_pool",palette = cat_pal,ax=ax,linewidth=0)

low_comps = sim_compare_df.loc[sim_compare_df.local<.23].loc[sim_compare_df.all_comps>.2].loc[sim_compare_df.same_pool].loc[sim_compare_df.pool.ne('trochanter_flexion')]
sns.scatterplot(data=low_comps, x="all_comps", y="local", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

hi_comps = sim_compare_df.loc[sim_compare_df.local>.5].loc[~sim_compare_df.same_pool]
sns.scatterplot(data=hi_comps, x="all_comps", y="local", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

# down_comps = sim_compare_df.loc[sim_compare_df.local>.4].loc[sim_compare_df.local<sim_compare_df.all_comps]
# sns.scatterplot(data=down_comps, x="all_comps", y="local", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

ax.plot([0, 1],[0,1],color='#000000')

fig.savefig('./figpanels/cosine_sim_local_v_all.svg',format='svg')

In [ ]:
cat_pal = {
    True: '#000000',
    False: '#DFDFDF',
    }

# pool_pal = {
#     'thorax_swing': '#A502AA',
#     'thorax_stance': '#00A2B4',
#     'trochanter_extension': '#D5CB6C',
#     'trochanter_flexion': '#3F42A2',
#     'femur_reductor': '#FF0000',
#     'tibia_extensor': '#CC8544',
#     'main_tibia_flexor': '#2E3191',
#     'auxiliary_tibia_flexor_B': '#2DB515',
#     'auxiliary_tibia_flexor_E': '#156005',
#     'ltm': '#FFF100',
#     'tarsus_depressor': '#ff81c0',
# }

fig, ax = plt.subplots( 1, 1, figsize=(12,12))
diff_comps = sim_compare_df.loc[~sim_compare_df.same_pool]
sns.scatterplot(data=diff_comps, x="local", y="desc", hue="same_pool",palette = cat_pal,ax=ax,linewidth=0)

same_comps = sim_compare_df.loc[sim_compare_df.same_pool]
sns.scatterplot(data=same_comps, x="local", y="desc", hue="same_pool",palette = cat_pal,ax=ax,linewidth=0)


hi_comps = sim_compare_df.loc[sim_compare_df.local>.5].loc[~sim_compare_df.same_pool]
sns.scatterplot(data=hi_comps, x="local", y="desc", hue="pool",palette = pool_pal,ax=ax,linewidth=0)
# low_comps = sim_compare_df.loc[sim_compare_df.local<.23].loc[sim_compare_df.same_pool].loc[sim_compare_df.pool.ne('trochanter_flexion')]
# sns.scatterplot(data=low_comps, x="all_comps", y="local", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

up_comps = sim_compare_df.loc[sim_compare_df.local<.2].loc[sim_compare_df.desc>sim_compare_df.local].loc[sim_compare_df.same_pool]
sns.scatterplot(data=up_comps, x="local", y="desc", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

# down_comps = sim_compare_df.loc[sim_compare_df.local>.4].loc[sim_compare_df.local<sim_compare_df.all_comps]
# sns.scatterplot(data=down_comps, x="all_comps", y="local", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

ax.plot([0, 1],[0,1],color='#000000')

fig.savefig('./figpanels/cosine_sim_desc_v_local.svg',format='svg')

In [ ]:
cat_pal = {
    True: '#000000',
    False: '#DFDFDF',
    }

# pool_pal = {
#     'thorax_swing': '#A502AA',
#     'thorax_stance': '#00A2B4',
#     'trochanter_extension': '#D5CB6C',
#     'trochanter_flexion': '#3F42A2',
#     'femur_reductor': '#FF0000',
#     'tibia_extensor': '#CC8544',
#     'main_tibia_flexor': '#2E3191',
#     'auxiliary_tibia_flexor_B': '#2DB515',
#     'auxiliary_tibia_flexor_E': '#156005',
#     'ltm': '#FFF100',
#     'tarsus_depressor': '#ff81c0',
# }

fig, ax = plt.subplots( 1, 1, figsize=(12,12))
diff_comps = sim_compare_df.loc[~sim_compare_df.same_pool]
sns.scatterplot(data=diff_comps, x="local", y="inter", hue="same_pool",palette = cat_pal,ax=ax,linewidth=0)

same_comps = sim_compare_df.loc[sim_compare_df.same_pool]
sns.scatterplot(data=same_comps, x="local", y="inter", hue="same_pool",palette = cat_pal,ax=ax,linewidth=0)


hi_comps = sim_compare_df.loc[sim_compare_df.local>.5].loc[~sim_compare_df.same_pool]
sns.scatterplot(data=hi_comps, x="local", y="inter", hue="pool",palette = pool_pal,ax=ax,linewidth=0)
# low_comps = sim_compare_df.loc[sim_compare_df.local<.23].loc[sim_compare_df.same_pool].loc[sim_compare_df.pool.ne('trochanter_flexion')]
# sns.scatterplot(data=low_comps, x="all_comps", y="local", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

up_comps = sim_compare_df.loc[sim_compare_df.local<.2].loc[sim_compare_df.desc>sim_compare_df.local].loc[sim_compare_df.same_pool]
sns.scatterplot(data=up_comps, x="local", y="inter", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

# down_comps = sim_compare_df.loc[sim_compare_df.local>.4].loc[sim_compare_df.local<sim_compare_df.all_comps]
# sns.scatterplot(data=down_comps, x="all_comps", y="local", hue="pool",palette = pool_pal,ax=ax,linewidth=0)

ax.plot([0, 1],[0,1],color='#000000')

fig.savefig('./figpanels/cosine_sim_inter_v_local.svg',format='svg')

# Rain cloud plots if within vs across pool cosine similarity

In [ ]:
all(sim_compare_df.lower_tri)
sim_compare_df

In [ ]:
(~sim_compare_df['same_pool']).sum()+sim_compare_df['same_pool'].sum()

In [ ]:
# f, ax = plt.subplots(figsize=(7, 7))
# # seaborn.violinplot(data=pprst, *, x=None, y=None, hue=None, order=None, hue_order=None, bw='scott', cut=2, scale='area', scale_hue=True, gridsize=100, width=0.8, inner='box', split=False, dodge=True, orient=None, linewidth=None, color=None, palette=None, saturation=0.75, ax=None, **kwargs)
# seaborn.violinplot(data=pprst,x='NT',y='pref_perc', orient='vertical',scale='count', cut=0, ax=ax)
# # sns.stripplot(data=pprst,x='NT',y='pref_perc',hue='syn_total',ax=ax)


# f, ax = plt.subplots(figsize=(7, 7))
# sns.violinplot(data=sim_compare_df,x='same_pool',y='all_comps',orient='vertical',scale='area', cut=0, ax=ax)

# sim_same_pool = sim_compare_df.loc[sim_compare_df.same_pool,:]
sim_same_pool_stacked = sim_compare_df.loc[:,['all_comps','pool','same_pool','lower_tri']]
sim_same_pool_stacked['comp_cat'] = 'all_comps'
sim_same_pool_stacked = sim_same_pool_stacked.rename({'all_comps':'cos_sim'},axis='columns')

block = sim_compare_df.loc[:,['local','pool','same_pool','lower_tri']]
block['comp_cat'] = 'local'
block = block.rename({'local':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

block = sim_compare_df.loc[:,['inter','pool','same_pool','lower_tri']]
block['comp_cat'] = 'inter'
block = block.rename({'inter':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

block = sim_compare_df.loc[:,['desc','pool','same_pool','lower_tri']]
block['comp_cat'] = 'desc'
block = block.rename({'desc':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])

block = sim_compare_df.loc[:,['asce','pool','same_pool','lower_tri']]
block['comp_cat'] = 'asce'
block = block.rename({'asce':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])

block = sim_compare_df.loc[:,['sense','pool','same_pool','lower_tri']]
block['comp_cat'] = 'sense'
block = block.rename({'sense':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

# ax.set_ylim([0,1.1])
# f.savefig('./figpanels/cosine_sim_violin_all_comps.svg',format='svg')

# ax.set_ylim([0,1.1])

# f.savefig('./figpanels/pool_pref_by_NT_violin.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.violinplot(data=sim_same_pool_stacked.loc[sim_same_pool_stacked.same_pool],x='comp_cat',y='cos_sim',orient='vertical',scale='area', cut=0, ax=ax,bw=.25)

f.savefig('./figpanels/cosine_sim_violin_same_pool.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.violinplot(data=sim_same_pool_stacked.loc[~sim_same_pool_stacked.same_pool,:],x='comp_cat',y='cos_sim',orient='vertical',scale='area', cut=0, ax=ax,bw=.25)

f.savefig('./figpanels/cosine_sim_violin_different_pools.svg',format='svg')


f, ax = plt.subplots(figsize=(7, 7))
sns.stripplot(data=sim_same_pool_stacked.loc[sim_same_pool_stacked.same_pool],x='comp_cat',y='cos_sim',ax=ax)

f.savefig('./figpanels/cosine_sim_rain_same_pool.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.stripplot(data=sim_same_pool_stacked.loc[~sim_same_pool_stacked.same_pool,:],x='comp_cat',y='cos_sim',ax=ax)

f.savefig('./figpanels/cosine_sim_rain_different_pools.svg',format='svg')

In [ ]:
# Stats

In [ ]:
sim_w_pool = sim_compare_df.loc[sim_compare_df['same_pool'],:]
sim_w_pool.melt(value_vars=['all_comps','local','desc','inter'])

In [ ]:
import scipy
scipy.stats.kruskal(sim_w_pool['all_comps'],
                    sim_w_pool['local'],
                    sim_w_pool['desc'],
                    sim_w_pool['inter'],)
# scipy.stats.kruskal(ratiodf.iloc[:,0],ratiodf.iloc[:,1],ratiodf.iloc[:,2],ratiodf.iloc[:,3],ratiodf.iloc[:,4],ratiodf.iloc[:,5],ratiodf.iloc[:,6],ratiodf.iloc[:,7],ratiodf.iloc[:,8])

In [ ]:
import scikit_posthocs 

scikit_posthocs.posthoc_conover(sim_w_pool.melt(value_vars=['all_comps','local','desc','inter']),val_col= 'value',group_col='variable',p_adjust='holm')


In [ ]:
# f, ax = plt.subplots(figsize=(7, 7))
# # seaborn.violinplot(data=pprst, *, x=None, y=None, hue=None, order=None, hue_order=None, bw='scott', cut=2, scale='area', scale_hue=True, gridsize=100, width=0.8, inner='box', split=False, dodge=True, orient=None, linewidth=None, color=None, palette=None, saturation=0.75, ax=None, **kwargs)
# seaborn.violinplot(data=pprst,x='NT',y='pref_perc', orient='vertical',scale='count', cut=0, ax=ax)
# # sns.stripplot(data=pprst,x='NT',y='pref_perc',hue='syn_total',ax=ax)


# f, ax = plt.subplots(figsize=(7, 7))
# sns.violinplot(data=sim_compare_df,x='same_pool',y='all_comps',orient='vertical',scale='area', cut=0, ax=ax)

# sim_same_pool = sim_compare_df.loc[sim_compare_df.same_pool,:]
sim_same_pool_stacked = sim_compare_df.loc[:,['all_comps','pool','same_pool','lower_tri']]
sim_same_pool_stacked['comp_cat'] = 'all_comps'
sim_same_pool_stacked = sim_same_pool_stacked.rename({'all_comps':'cos_sim'},axis='columns')

block = sim_compare_df.loc[:,['local','pool','same_pool','lower_tri']]
block['comp_cat'] = 'local'
block = block.rename({'local':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

block = sim_compare_df.loc[:,['inter','pool','same_pool','lower_tri']]
block['comp_cat'] = 'long_range'
block = block.rename({'inter':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

block = sim_compare_df.loc[:,['desc','pool','same_pool','lower_tri']]
block['comp_cat'] = 'long_range'
block = block.rename({'desc':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

# ax.set_ylim([0,1.1])
# f.savefig('./figpanels/cosine_sim_violin_all_comps.svg',format='svg')

# ax.set_ylim([0,1.1])

# f.savefig('./figpanels/pool_pref_by_NT_violin.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.violinplot(data=sim_same_pool_stacked.loc[sim_same_pool_stacked.same_pool],x='comp_cat',y='cos_sim',orient='vertical',scale='area', cut=0, ax=ax,bw=.25)

f.savefig('./figpanels/cosine_sim_violin_same_pool_long.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.violinplot(data=sim_same_pool_stacked.loc[~sim_same_pool_stacked.same_pool,:],x='comp_cat',y='cos_sim',orient='vertical',scale='area', cut=0, ax=ax, bw=.3)

f.savefig('./figpanels/cosine_sim_violin_different_pools_long.svg',format='svg')


f, ax = plt.subplots(figsize=(7, 7))
sns.stripplot(data=sim_same_pool_stacked.loc[sim_same_pool_stacked.same_pool],x='comp_cat',y='cos_sim',ax=ax)

f.savefig('./figpanels/cosine_sim_rain_same_pool_long.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.stripplot(data=sim_same_pool_stacked.loc[~sim_same_pool_stacked.same_pool,:],x='comp_cat',y='cos_sim',ax=ax)

f.savefig('./figpanels/cosine_sim_rain_different_pools_long.svg',format='svg')

In [ ]:
# fig, axes = plt.subplots()

# # Seaborn violin plot
# sns.violinplot(data=a, color="#af52f4", inner=None, linewidth=0, saturation=0.5)

# # Normal boxplot has full range, same in Seaborn boxplot
# axes.boxplot(a, whis='range', positions=np.array([0]),
#             showcaps=False,widths=0.06, patch_artist=True,
#             boxprops=dict(color="indigo", facecolor="indigo"),
#             whiskerprops=dict(color="indigo", linewidth=2),
#             medianprops=dict(color="w", linewidth=2 ))

# axes.set_xlim(-1,1)
# plt.show()

In [ ]:
sim_compare_df

# Stats

In [ ]:
# Simple version, just mean and std of same pool, not same pool
x = sim_compare_df.loc[sim_compare_df.same_pool,'local'].mean()
y = sim_compare_df.loc[sim_compare_df.same_pool,'inter'].mean()

x2 = sim_compare_df.loc[~sim_compare_df.same_pool,'local'].mean()
y2 = sim_compare_df.loc[~sim_compare_df.same_pool,'inter'].mean()


In [ ]:
import scipy

scipy.stats.ttest_rel(sim_compare_df.loc[sim_compare_df.same_pool,'local'],sim_compare_df.loc[sim_compare_df.same_pool,'all_comps'],alternative='two-sided',axis=0)

In [ ]:
sim_compare_df.loc[~sim_compare_df.same_pool,'local'].to_numpy()

# Local Femur

In [ ]:
# Cluster according to everything but fragments
All = slice(None)
mn_mi = pre_to_mn_df.loc['local',('L','Leg',['femur','tibia'],All,['tibia_extensor','main_tibia_flexor','auxiliary_tibia_flexor','tarsus_unidentified',],All,All)].columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['local',('L','Leg',['femur','tibia'],All,['tibia_extensor','main_tibia_flexor','auxiliary_tibia_flexor','tarsus_unidentified',],All,All)].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_femur_local_grayscale.svg',format='svg')

In [ ]:
sim_mat_all = cosine_similarity(pre_to_mn_df.to_numpy().transpose())
sim_mat_local = cosine_similarity(pre_to_mn_df.loc['local',:].to_numpy().transpose())
sim_mat_desc = cosine_similarity(pre_to_mn_df.loc['descending',:].to_numpy().transpose())
sim_mat_inter = cosine_similarity(pre_to_mn_df.loc['intersegmental',:].to_numpy().transpose())

sim_mat_local

## Descending Femur

In [ ]:
# Cluster according to everything but fragments
All = slice(None)
mn_mi = pre_to_mn_df.loc['descending',('L','Leg',['femur','tibia'],All,['tibia_extensor','main_tibia_flexor','auxiliary_tibia_flexor','tarsus_unidentified',],All,All)].columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['descending',('L','Leg',['femur','tibia'],All,['tibia_extensor','main_tibia_flexor','auxiliary_tibia_flexor','tarsus_unidentified',],All,All)].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

# fig.savefig('./figpanels/cos_sim_femur_local_grayscale.svg',format='svg')

fig.savefig('./figpanels/cos_sim_femur_descending_grayscale.svg',format='svg')

## Intersegmental Femur

In [ ]:
# Cluster according to everything but fragments
All = slice(None)
mn_mi = pre_to_mn_df.loc['intersegmental',('L','Leg',['femur','tibia'],All,['tibia_extensor','main_tibia_flexor','auxiliary_tibia_flexor','tarsus_unidentified',],All,All)].columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['intersegmental',('L','Leg',['femur','tibia'],All,['tibia_extensor','main_tibia_flexor','auxiliary_tibia_flexor','tarsus_unidentified',],All,All)].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

# fig.savefig('./figpanels/cos_sim_femur_local_grayscale.svg',format='svg')
# fig.savefig('./figpanels/cos_sim_femur_descending_grayscale.svg',format='svg')
fig.savefig('./figpanels/cos_sim_femur_intersegmental_grayscale.svg',format='svg')

## Intersegmental+descending Femur

In [ ]:
# Cluster according to everything but fragments
All = slice(None)
mn_mi = pre_to_mn_df.loc[['intersegmental','descending'],('L','Leg',['femur','tibia'],All,['tibia_extensor','main_tibia_flexor','auxiliary_tibia_flexor','tarsus_unidentified',],All,All)].columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc[['intersegmental','descending'],('L','Leg',['femur','tibia'],All,['tibia_extensor','main_tibia_flexor','auxiliary_tibia_flexor','tarsus_unidentified',],All,All)].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

# fig.savefig('./figpanels/cos_sim_femur_local_grayscale.svg',format='svg')
# fig.savefig('./figpanels/cos_sim_femur_descending_grayscale.svg',format='svg')
fig.savefig('./figpanels/cos_sim_femur_longrange_grayscale.svg',format='svg')

## Ascending Femur

In [ ]:
# Cluster according to everything but fragments
All = slice(None)
mn_mi = pre_to_mn_df.loc['ascending',('L','Leg','femur',All,All,All,All)].columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['ascending',('L','Leg','femur',All,All,All,All)].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_femur_ascending_grayscale.svg',format='svg')

# Plot number of local synapses vs descending inputs

In [ ]:
fig = plt.figure(1, figsize=(10, 6))
ax = plt.subplot2grid((1,1),(0,0))
ax.set_ylim([0, 2000])
ax.set_xlim([0, 9000])
ax.set_ylabel('Number of Descending inputs')
ax.set_xlabel('Number of Local inputs')
# ax.set_xscale("log");

x = pre_to_mn_df.loc['local',mpool_dict['trochanter_flexion']].sum()
y = pre_to_mn_df.loc['descending',mpool_dict['trochanter_flexion']].sum()
ax.scatter(x,y,color='#2E3191',marker='^')
y = pre_to_mn_df.loc['intersegmental',mpool_dict['trochanter_flexion']].sum()
# ax.scatter(x,y,color='#2E3191',marker='^')

x = pre_to_mn_df.loc['local',mpool_dict['trochanter_extension']].sum()
y = pre_to_mn_df.loc['descending',mpool_dict['trochanter_extension']].sum()
ax.scatter(x,y,color='#F6921E',marker='^')
y = pre_to_mn_df.loc['intersegmental',mpool_dict['trochanter_extension']].sum()
# ax.scatter(x,y,color='#2E3191',marker='^')

x = pre_to_mn_df.loc['local',mpool_dict['main_tibia_flexor']].sum()
y = pre_to_mn_df.loc['descending',mpool_dict['main_tibia_flexor']].sum()
ax.scatter(x,y,color='#2E3191')
y = pre_to_mn_df.loc['intersegmental',mpool_dict['main_tibia_flexor']].sum()
# ax.scatter(x,y,color='#2E3191',marker='^')

x = pre_to_mn_df.loc['local',mpool_dict['tibia_extensor']].sum()
y = pre_to_mn_df.loc['descending',mpool_dict['tibia_extensor']].sum()
ax.scatter(x,y,color='#F6921E')
y = pre_to_mn_df.loc['intersegmental',mpool_dict['tibia_extensor']].sum()
# ax.scatter(x,y,color='#F6921E',marker='^')

x = pre_to_mn_df.loc['local',mpool_dict['auxiliary_tibia_flexor_B']].sum()
y = pre_to_mn_df.loc['descending',mpool_dict['auxiliary_tibia_flexor_B']].sum()
ax.scatter(x,y,color='#2DB515')
y = pre_to_mn_df.loc['intersegmental',mpool_dict['auxiliary_tibia_flexor_B']].sum()
# ax.scatter(x,y,color='#2DB515',marker='^')

x = pre_to_mn_df.loc['local',mpool_dict['auxiliary_tibia_flexor_E']].sum()
y = pre_to_mn_df.loc['descending',mpool_dict['auxiliary_tibia_flexor_E']].sum()
ax.scatter(x,y,color='#2DB515')
y = pre_to_mn_df.loc['intersegmental',mpool_dict['auxiliary_tibia_flexor_E']].sum()
# ax.scatter(x,y,color='#2DB515',marker='^')
fig.savefig('./figpanels/descending_vs_local_inputs.svg',format='svg')

In [ ]:
mpool_dict['main_tibia_flexor']

In [ ]:
pre_to_mn_df.loc['local',mpool_dict['tibia_extensor']].sum()

# Find a couple of interesting premotor neurons that innervate the flexor pool
Plot their inputs to the flexor pool    

In [ ]:
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

mft_df = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['main_tibia_flexor']]
mft_df = mft_df.iloc[0:50,:]

mft_df_mtf = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['main_tibia_flexor_muscle']]
mft_df_mtf = mft_df_mtf.iloc[0:50,:]

ratiodf = mft_df_mtf.divide(mft_df_mtf.sum(axis=1),axis='index')
ratiodf.replace([np.inf, -np.inf], np.nan, inplace=True)
ratiodf = ratiodf.dropna()

In [ ]:
mft_df.loc[(All,All,'E',All)]
mft_df.sum(axis=1)
# mft_df[ratiodf.apply(lambda x: x.is_monotonic,axis=1)]

In [ ]:
cool_local_pMN1 = mft_df.loc[('main_tibia_flexor','Glu','B','21A',All)]
cool_local_pMN1_id = cool_local_pMN1.index.get_level_values('segID')[0]
cool_local_pMN1_id

In [ ]:
ratiodf

In [ ]:
cool_local_pMN2 = mft_df.loc[ratiodf.apply(lambda x: x.is_monotonic,axis=1)]
cool_local_pMN2
cool_local_pMN2_id = cool_local_pMN2.index.get_level_values('segID')[0]
cool_local_pMN2_id

In [ ]:
cool_local_pMN2

In [ ]:
cool_local_pMN3 = mft_df.loc[('main_tibia_flexor','Ach','C','3A',All)]
cool_local_pMN3
cool_local_pMN3_id = cool_local_pMN3.index.get_level_values('segID')[2]
cool_local_pMN3_id

In [ ]:
cool_local_pMN4 = mft_df.loc[('main_tibia_flexor','Glu','R','R1',All)]
cool_local_pMN4_id = cool_local_pMN4.index.get_level_values('segID')[0]
cool_local_pMN4

In [ ]:
def build_synapses_onto_flexor_mns(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_points,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url


def build_spheres_onto_flexor_mns(render_neurons,syn_df_dict,clr_dict): # #F6921E

    img_layer = ngstbld.ImageLayerConfig(name='fanc_v4',
                                 source=client.info.image_source(),
                                 )
    seg_layer = ngstbld.SegmentationLayerConfig(name = 'seg',
                                        source =  client.info.segmentation_source(),
                                        fixed_ids = render_neurons)

    sb1 = ngstbld.StateBuilder(layers=[img_layer, seg_layer], client=client, resolution=[4.3,4.3,45])
    state_builders = [sb1]

    df_list = [None]

    for key in syn_df_dict.keys():
        print('layer {}'.format(key))
        # pool_points = ngstbld.PointMapper(point_column='pre_pt_position') ######################## change this to toggle rendering of pre- or post- synaptic points
        pool_spheres = ngstbld.SphereMapper(center_column='pre_pt_position', radius_column='radius')
        pool_layer = ngstbld.AnnotationLayerConfig(name=key,
                                        mapping_rules=pool_spheres,
                                        linked_segmentation_layer=seg_layer.name,
                                        color=clr_dict[key] )
        sb_pool = ngstbld.StateBuilder(layers=[pool_layer],client=client)
        state_builders.append(sb_pool)
        df_list.append(syn_df_dict[key])

    sb=ngstbld.ChainedStateBuilder(statebuilders=state_builders)
    # state = sb.render_state(data_list=[ach_df,gaba_df,glu_df],return_as='dict')

    print('{} sbs, {} dfs'.format(len(state_builders),len(df_list)))

    state = sb.render_state(data_list=df_list,return_as='dict')
    state_id = client.state.upload_state_json(state)

    DEFAULT_NGL = "https://neuromancer-seung-import.appspot.com/"

    url = client.state.build_neuroglancer_url(state_id, ngl_url=DEFAULT_NGL)

    return url

In [ ]:
timestamp = connectome_create.get_timestamp()

flexor_rnk = mft_df.columns.get_level_values('rank').to_list()
flexor_rnk[1] = '1'
flexor_id = mft_df.columns.get_level_values('segID').to_list()

for cool_pmn in [cool_local_pMN1_id,cool_local_pMN2_id,cool_local_pMN3_id,cool_local_pMN4_id]:
    render_neurons = [cool_pmn]

    syn_df_dict = {}
    clr_dict = {}
    clr_list = ['#AAAAFF','#6666FF','#4444CC','#2222AA','#0000AA']
    for rnk,id,clr in zip(flexor_rnk,flexor_id,clr_list):
        syn_df = client.materialize.synapse_query(post_ids = id, pre_ids=cool_pmn, timestamp=timestamp)
        syn_df['radius'] = 180
        syn_df_dict[rnk] = syn_df
        clr_dict[rnk] = clr

    # print(build_synapses_onto_flexor_mns(render_neurons,syn_df_dict,clr_dict))
    print(build_spheres_onto_flexor_mns(render_neurons,syn_df_dict,clr_dict))

# Find a couple of interesting FETi neurons

In [ ]:
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

eti_df = femur_df_local.loc[('tibia_extensor',All,All),mpool_dict['tibia_extensor']]
eti_df = eti_df.iloc[0:50,:]
# eti_df.index.get_level_values('segID').to_list()
# eti_df


In [ ]:
# 1 A nice looking 8A neuron that makes a lot of synapses (Glu)
cool_local_eti_pmn_1 = eti_df.loc[('tibia_extensor','Glu','S','8A_')].index.get_level_values('segID')[0]
cool_local_eti_pmn_1

In [ ]:
# 1 A nice looking 8A neuron that makes a lot of synapses (Glu)
cool_local_eti_pmn_2 = eti_df.loc[('tibia_extensor','Ach','K','4B')].index.get_level_values('segID')[-2]
cool_local_eti_pmn_2

In [ ]:
timestamp = connectome_create.get_timestamp()

flexor_rnk = eti_df.columns.get_level_values('rank').to_list()
flexor_id = eti_df.columns.get_level_values('segID').to_list()


for cool_pmn in [cool_local_eti_pmn_1,cool_local_eti_pmn_2 ]:
    render_neurons = [cool_pmn]

    syn_df_dict = {}
    clr_dict = {}
    clr_list = ['#AAAAFF','#6666FF','#4444CC','#2222AA','#0000AA']
    for rnk,id,clr in zip(flexor_rnk,flexor_id,clr_list):
        syn_df = client.materialize.synapse_query(post_ids = id, pre_ids=cool_pmn, timestamp=timestamp)
        syn_df['radius'] = 180
        syn_df_dict[rnk] = syn_df
        clr_dict[rnk] = clr

    # print(build_synapses_onto_flexor_mns(render_neurons,syn_df_dict,clr_dict))
    print(build_spheres_onto_flexor_mns(render_neurons,syn_df_dict,clr_dict))

# Look at a small snippet of the full matrix

In [ ]:
import utils
muscle_tuple_dict = utils.get_motor_pool_tuple_dict()
mpool_dict = utils.get_motor_pool_tuple_dict()
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

mft_df = femur_df_local.loc[(['tibia_extensor','main_tibia_flexor'],All,All),mpool_dict['tibia_extensor_flexorA']]
mft_df = femur_df_local.loc[(['tibia_extensor','main_tibia_flexor'],All,All),mpool_dict['tibia_extensor_mainflexor_auxA']]
# mft_df = mft_df.iloc[0:50,:]

fig = plt.figure(1, figsize = [6,6])
lbls = utils.mn_labels(mft_df.columns.to_frame(),depth='rank')
ax = sns.heatmap(np.log10(mft_df.to_numpy()+1), xticklabels=lbls,cmap=utils.white_dense())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)

plt.savefig('./figpanels/tibiaflexA_heat_map.svg',format='svg')
plt.show()

In [ ]:
mpool_dict['tibia_extensor_flexorA']

# Compare ratio of inputs (Tibia extension)

In [ ]:
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

ext_df = femur_df_local.loc[('tibia_extensor',All,All),mpool_dict['tibia_extensor']]
ext_df = ext_df.iloc[0:50,:]

s = ext_df.sum(axis='columns')

ext_df = ext_df.loc[s.sort_values(ascending=False).index,:]
ext_df


In [ ]:

fig = plt.figure(1, figsize = [6,6])
lbls = utils.mn_labels(ext_df.columns.to_frame(),depth='rank')
ax = sns.heatmap(ext_df.to_numpy(), xticklabels=lbls,cmap=utils.white_dense())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()
plt.savefig('./figpanels/tibiaextensor_heat_map.svg',format='svg')

ratiodf = ext_df.divide(ext_df.sum(axis=1),axis='index')
ratiodf.replace([np.inf, -np.inf], np.nan, inplace=True)
ratiodf = ratiodf.dropna()

x = [i for i in range(ext_df.shape[1])]
fig, ax = plt.subplots( 1, 1, figsize=(8,8))
ax.plot(x,ratiodf.to_numpy().T,color='#CCCCCC',marker='.')
ax.plot(x,np.median(ratiodf.to_numpy(),axis=0),color='#000000',marker='.',ls='')

ratiodf.columns = ratiodf.columns.droplevel(['side','nerve','segment','function','muscle','segID'])
ranks = ratiodf.copy()
for c in range(ranks.shape[1]):
    ranks.iloc[:,c] = c


df = pd.concat([pd.concat([ranks.iloc[:,x] for x in range(ranks.shape[1])]),pd.concat([ratiodf.iloc[:,x] for x in range(ratiodf.shape[1])])],axis=1)
df.columns=['rank','ratio']
df['total'] = ext_df.sum(axis=1)
# df['total'] = np.log10(mft_df.sum(axis=1)+1)

dx = "rank"; dy = "ratio"; ort = "v"; pal = "Set2"; sigma = .2
f, ax = plt.subplots(figsize=(32, 7))

ax=pt.RainCloud(x = dx, y = dy, data = df, palette = pal, bw = sigma,
                 width_viol = .6, ax = ax, orient = ort, move = .2,pointplot=True)

plt.title("Relative of synapse counts compared to Fast MN")

ax.set_ylim([-.1,1.1])
plt.savefig('./figpanels/tibia_ext_prop_conn_raincloud.svg')


fig, ax = plt.subplots( 1, 1, figsize=(20,8))
sns.color_palette("YlOrBr", as_cmap=True)
sns.scatterplot(data=df.iloc[::-1], x="rank", y="ratio", hue="total",palette=utils.white_dense(),ax=ax)
# ax = sns.scatterplot(data=tips, x="sex", y="total_bill", hue="total_bill", palette="viridis", legend=False)
pts = ax.collections[0]
pts.set_offsets(pts.get_offsets() + np.c_[np.random.uniform(-.2, .2, len(df)), np.zeros(len(df))])
ax.margins(x=.5)
ax.autoscale_view()
# ax.get_legend().remove()
norm = plt.Normalize(0,df['total'].max())
sm = plt.cm.ScalarMappable(cmap=utils.white_dense(), norm=norm)
sm.set_array([])
ax.figure.colorbar(sm)

                # [top 4B, other 4B, big old 8A, one exception]
for cool_pmn in [648518346490338058,648518346490367836,648518346508809279	,648518346472455883]:
    pmndf = df.loc[(All,All,All,All,cool_pmn)].copy().reset_index()
    sns.lineplot(data=pmndf,x="rank",y='ratio')

ax.set_ylim([-.1,1.1])
plt.savefig('./figpanels/tibia_ext_prop_conn_stripplot.svg')


In [ ]:
a = np.median(ratiodf.to_numpy(),axis=0)
a[1]/a[0]

# Compare ratio of inputs (Flexors)

In [ ]:
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

mft_df = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['aux_main_tibia_flexor']]
mft_df = mft_df.iloc[0:50,:]

fig = plt.figure(1, figsize = [6,6])
lbls = utils.mn_labels(mft_df.columns.to_frame(),depth='rank')
ax = sns.heatmap(mft_df.to_numpy(), xticklabels=lbls,cmap=utils.white_dense())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()
plt.savefig('./figpanels/tibiaflexA_heat_map.svg',format='svg')

import ptitprince as pt

ratiodf = mft_df.divide(mft_df.sum(axis=1),axis='index')
ratiodf.replace([np.inf, -np.inf], np.nan, inplace=True)
ratiodf = ratiodf.dropna()

x = [i for i in range(mft_df.shape[1])]

fig, ax = plt.subplots( 1, 1, figsize=(8,8))
ax.plot(x,ratiodf.to_numpy().T,color='#CCCCCC',marker='.')

ax.plot(x,np.median(ratiodf.to_numpy(),axis=0),color='#000000',marker='.',ls='')

# total = mft_df.sum(axis=0)
# total = total/total.sum()
# ax.plot(x,total.to_numpy(),color='#FF0000',marker='.',ls='')

ax.set_xlabel('Rank of tibia flexor MN')
ax.set_ylabel('Ratio of input, compare to Fast flexor')

ratiodf.columns = ratiodf.columns.droplevel(['side','nerve','segment','function','muscle','segID'])
ranks = ratiodf.copy()
for c in range(ranks.shape[1]):
    ranks.iloc[:,c] = c

df = pd.concat([pd.concat([ranks.iloc[:,x] for x in range(ranks.shape[1])]),pd.concat([ratiodf.iloc[:,x] for x in range(ratiodf.shape[1])])],axis=1)
df.columns=['rank','ratio']
df['total'] = mft_df.sum(axis=1)
# df['total'] = np.log10(mft_df.sum(axis=1)+1)

dx = "rank"; dy = "ratio"; ort = "v"; pal = "Set2"; sigma = .2
f, ax = plt.subplots(figsize=(32, 7))

ax=pt.RainCloud(x = dx, y = dy, data = df, palette = pal, bw = sigma,
                 width_viol = .6, ax = ax, orient = ort, move = .2,pointplot=True)

plt.title("Relative of synapse counts compared to Fast MN")

ax.set_ylim([-.1,.75])
plt.savefig('./figpanels/tibflexA_prop_conn_raincloud.svg')

# fig, ax = plt.subplots( 1, 1, figsize=(20,8))
# sns.swarmplot(data=df, x="rank", y="ratio", hue="total",palette="binary",ax=ax)
# sns.stripplot(data=df, x="rank", y="ratio", hue="total",palette="binary",ax=ax)# ax.get_legend().remove()
# ax.set_ylim([-.1,.75])
# plt.savefig('./figpanels/tibflexA_prop_conn_stripplot.svg')


In [ ]:

fig, ax = plt.subplots( 1, 1, figsize=(20,8))
sns.scatterplot(data=df.iloc[::-1], x="rank", y="ratio", hue="total",palette=utils.white_dense(),ax=ax)
# ax = sns.scatterplot(data=tips, x="sex", y="total_bill", hue="total_bill", palette="viridis", legend=False)
pts = ax.collections[0]
pts.set_offsets(pts.get_offsets() + np.c_[np.random.uniform(-.2, .2, len(df)), np.zeros(len(df))])
ax.margins(x=.5)
ax.autoscale_view()
ax.get_legend().remove()
norm = plt.Normalize(0,df['total'].max())
sm = plt.cm.ScalarMappable(cmap=utils.white_dense(), norm=norm)
sm.set_array([])
ax.figure.colorbar(sm)

# for cool_pmn in [cool_local_pMN1_id,cool_local_pMN2_id,cool_local_pMN3_id,cool_local_pMN4_id]:
#     pmndf = df.loc[(All,All,All,All,cool_pmn)].copy().reset_index()
#     sns.lineplot(data=pmndf,x="rank",y='ratio')

ax.set_ylim([-.1,.75])
plt.savefig('./figpanels/tibiaflexA_prop_conn_stripplot.svg')


In [ ]:
# df.loc[(All,All,All,All,cool_pmn)]
pmndf.reset_index()

In [ ]:
[cool_local_pMN1_id,cool_local_pMN2_id,cool_local_pMN3_id,cool_local_pMN4_id]

In [ ]:
ratiodf

In [ ]:
import scipy

scipy.stats.mannwhitneyu(ratiodf.iloc[:,7],ratiodf.iloc[:,8],alternative='two-sided',)

In [ ]:
scipy.stats.kruskal(ratiodf.iloc[:,4],ratiodf.iloc[:,5],ratiodf.iloc[:,6],ratiodf.iloc[:,7],ratiodf.iloc[:,8])
# scipy.stats.kruskal(ratiodf.iloc[:,0],ratiodf.iloc[:,1],ratiodf.iloc[:,2],ratiodf.iloc[:,3],ratiodf.iloc[:,4],ratiodf.iloc[:,5],ratiodf.iloc[:,6],ratiodf.iloc[:,7],ratiodf.iloc[:,8])

In [ ]:
import scikit_posthocs 

scikit_posthocs.posthoc_conover(df,val_col= 'ratio',group_col='rank',p_adjust='holm')


## Just the main tibia flexors

In [ ]:
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

mft_df = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['main_tibia_flexor_muscle']]
mft_df = mft_df.iloc[0:50,:]

fig = plt.figure(1, figsize = [6,6])
lbls = utils.mn_labels(mft_df.columns.to_frame(),depth='rank')
ax = sns.heatmap(mft_df.to_numpy(), xticklabels=lbls,cmap=utils.white_dense())
cbar = ax.collections[0].colorbar
cbar.set_label(label = '# of synapses',)
plt.savefig('./figpanels/mtf_heat_map.svg',format='svg')
plt.show()


import ptitprince as pt

ratiodf = mft_df.divide(mft_df.sum(axis=1),axis='index')
ratiodf.replace([np.inf, -np.inf], np.nan, inplace=True)
ratiodf = ratiodf.dropna()

x = [i for i in range(mft_df.shape[1])]

fig, ax = plt.subplots( 1, 1, figsize=(8,8))
ax.plot(x,ratiodf.to_numpy().T,color='#CCCCCC',marker='.')

ax.plot(x,np.median(ratiodf.to_numpy(),axis=0),color='#000000',marker='.',ls='')

# total = mft_df.sum(axis=0)
# total = total/total.sum()
# ax.plot(x,total.to_numpy(),color='#FF0000',marker='.',ls='')

ax.set_xlabel('Rank of tibia flexor MN')
ax.set_ylabel('Ratio of input, compare to Fast flexor')

ratiodf.columns = ratiodf.columns.droplevel(['side','nerve','segment','function','muscle','segID'])
ranks = ratiodf.copy()
for c in range(ranks.shape[1]):
    ranks.iloc[:,c] = c

df = pd.concat([pd.concat([ranks.iloc[:,x] for x in range(ranks.shape[1])]),pd.concat([ratiodf.iloc[:,x] for x in range(ratiodf.shape[1])])],axis=1)
df.columns=['rank','ratio']
df['total'] = mft_df.sum(axis=1)
df['total'] = np.log10(mft_df.sum(axis=1)+1)

dx = "rank"; dy = "ratio"; ort = "v"; pal = "Set2"; sigma = .2
f, ax = plt.subplots(figsize=(32, 7))

ax=pt.RainCloud(x = dx, y = dy, data = df, palette = pal, bw = sigma,
                 width_viol = .6, ax = ax, orient = ort, move = .2)
ax.set_ylim([-.1,.75])
plt.title("Ratio of synapse counts compared to Fast MN")
plt.savefig('tibflexA_heat_map.svg')
plt.savefig('./figpanels/mtf_prop_conn_raincloud.svg')

fig, ax = plt.subplots( 1, 1, figsize=(20,8))
sns.scatterplot(data=df.iloc[::-1], x="rank", y="ratio", hue="total",palette=utils.white_dense(),ax=ax)
# ax = sns.scatterplot(data=tips, x="sex", y="total_bill", hue="total_bill", palette="viridis", legend=False)
pts = ax.collections[0]
pts.set_offsets(pts.get_offsets() + np.c_[np.random.uniform(-.2, .2, len(df)), np.zeros(len(df))])
ax.margins(x=.5)
ax.autoscale_view()
ax.get_legend().remove()
norm = plt.Normalize(0,df['total'].max())
sm = plt.cm.ScalarMappable(cmap=utils.white_dense(), norm=norm)
sm.set_array([])
ax.figure.colorbar(sm)

ax.set_ylim([-.1,.75])
plt.savefig('./figpanels/mtf_prop_conn_stripplot.svg')


In [ ]:
len(df)

In [ ]:
import scipy

scipy.stats.mannwhitneyu(ratiodf.iloc[:,0],ratiodf.iloc[:,4],alternative='less',)

In [ ]:
scipy.stats.kruskal(ratiodf.iloc[:,0],ratiodf.iloc[:,1],ratiodf.iloc[:,2],ratiodf.iloc[:,3],ratiodf.iloc[:,4])

In [ ]:
scikit_posthocs.posthoc_conover(df,val_col= 'ratio',group_col='rank',p_adjust='holm')


In [ ]:
df[ratiodf.apply(lambda x: x.is_monotonic,axis=1)]

# Bootstrap the cosine similarity metrics
There are a number of ways to make these comparisons. We can consider both which weights are shuffled, and what the metric of comparison is. 

Metrics
1. Compute the average similarity within the pool, project each neuron onto the average. What I have done so far, gives a single number.
2. Compute the pairwise similarity between each neuron. Then what? Compute mean and variance?

Shuffling
1. I could shuffle only the weights that a particular preMN makes onto the pool, i.e. just in the small square. That preserves density but mixes up propotionality.
2. I could shuffle the weights of all the connections that a hemilineage makes within the preferred pool. That preserve density, but shuffles the propotionality a bit more. 
3. I could shuffle all of the weights of a given premotor neuron, across pools. That preserves neither density nor proportionality. 
4. I could shuffle the weights of all the connection from a hemilineages onto the pools, including preMNs that don't prefer the pool. That also preserves neither density nor propotionality.
Start with local neurons onto extensors and flexors

## Plotting functions

In [ ]:
def plot_sim_map(sim_mat,df,fn=None):
    fig = plt.figure(1, figsize = [6,6])

    mn_mi = df.columns.to_frame()
    ax = sns.heatmap(sim_mat, xticklabels=mn_mi.loc[:,'rank'].to_list(),cmap=plt.get_cmap('Greys'),vmin=0,vmax=1)
    ax.xaxis.set_ticks_position('top')
    cbar = ax.collections[0].colorbar
    ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
    cbar.set_label(label = 'cosine similarity')
    plt.xlabel('MTF')
    plt.show()

    if not fn is None:
        fig.savefig('./figpanels/{}.svg'.format(fn),format='svg')

def plot_conn_map(conn_mat,df,fn=None):
    fig = plt.figure(2, figsize = [6,12])
    mn_mi = df.columns.to_frame()
    ax = sns.heatmap(conn_mat, xticklabels=mn_mi.loc[:,'rank'].to_list(), cmap=utils.white_dense())
    # ax.xaxis.set_ticks_position('top')
    cbar = ax.collections[0].colorbar
    ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

    cbar.set_label(label = '# syn')
    plt.xlabel('Motor Neurons')
    plt.show()
    if not fn is None:
        fig.savefig('./figpanels/{}.svg'.format(fn),format='svg')

def plot_hem_conn_map(conn_mat,df,ct,fn=None):
    fig = plt.figure(2, figsize = [6,12])
    mn_mi = df.columns.to_frame()
    ax = sns.heatmap(conn_mat, xticklabels=mn_mi.loc[:,'rank'].to_list(), cmap=utils.white_dense())
    # ax.xaxis.set_ticks_position('top')
    cbar = ax.collections[0].colorbar
    ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

    yticks = df.index.get_locs((All,All,All,All,ct,All))
    ax.set_yticks(yticks)
    ax.set_yticklabels([ct]*len(yticks))
    
    cbar.set_label(label = '# syn')
    plt.xlabel('Motor Neurons')
    plt.show()

    if not fn is None:
        fig.savefig('./figpanels/{}.svg'.format(fn),format='svg')


def plot_mask(conn_mat,df,fn=None):
    fig = plt.figure(3, figsize = [6,12])
    mn_mi = df.columns.to_frame()
    ax = sns.heatmap(conn_mat, xticklabels=mn_mi.loc[:,'rank'].to_list(),cmap=plt.get_cmap('Greys'),vmin=0,vmax=1)
    # ax.xaxis.set_ticks_position('top')
    cbar = ax.collections[0].colorbar
    ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

    cbar.set_label(label = '# syn')
    plt.xlabel('Motor Neurons')
    plt.show()
    
    if not fn is None:
        print(fn)
        fig.savefig('./figpanels/{}.svg'.format(fn),format='svg')

def plot_shuf_cum(x_unsorted,color = '#000000',ref_x=None, fn=None,fignum=4,title=None):
    fig = plt.figure(fignum, figsize=(10, 6))
    ax = plt.subplot2grid((1,1),(0,0))
    ax.set_ylim([-.1, 1.1])
    ax.set_xlim([.945, .96])
    ax.set_xlim([0, 1])
    ax.set_ylabel('Cummulative fraction')
    ax.set_xlabel('Sum of cosine similarity for MTF')
    ax.set_title(title)

    # colors = {1: '#CCCCCC', 2: '#666666',3: '#000000'}
    # for m in [1,2,3]:
    # exec('mu=mixups_{}'.format(m))
    y = np.array(range(len(x_unsorted))).astype(float)/len(x_unsorted)
    ax.plot(sorted(x_unsorted),y,color=color)
    if not ref_x is None:
        ax.plot(ref_x*np.array([1,1]),np.array([0,1]),color='#0000FF')

    pct_below_actual = (x_unsorted<ref_x).sum() / len(x_unsorted)
    p=1-pct_below_actual
    ax.text(.94,.5,'p = {:.5f}'.format(p))

    if not fn is None:
        fig.savefig('./figpanels/{}.eps'.format(fn),format='eps')


## Bootstrap functions

In [ ]:
def get_hat_bar_from_numpy(pmn_conn_mat):
    norm = pmn_conn_mat.copy()
    for c in range(pmn_conn_mat.shape[1]):
        sq = pmn_conn_mat[:,c]**2
        norm[:,c] = np.sqrt(sq.sum())
    hats = pmn_conn_mat/norm

    hat_bar = hats.mean(axis=1)
    hat_bar = hat_bar/np.sqrt((hat_bar**2).sum())
    return hat_bar, hats, pmn_conn_mat 

def residue_norms(hat_bar,mat):
    proj = np.matmul(mat.T,hat_bar)
    residues = mat
    for c in range(residues.shape[1]):
        residues[:,c] = mat-(hat_bar*proj[c])
    residue_norms = np.sqrt((residues**2).sum(axis=0))
    residue_R1_norms = (np.abs(residues)).sum(axis=0)
    return residue_norms,residue_R1_norms

def mix_up_under_mask(mat_in,mask=None,rng=None):
    if mask is None:
        mask = np.full(mat_in.shape, True)
    if rng is None:
        rng = np.random.default_rng()
    
    mat_out = mat_in.copy()
    mat_out[mask] = rng.permutation(mat_in[mask])
    return mat_out


def choice_under_mask(mat_in,mask=None,rng=None,replace=True):
    if mask is None:
        mask = np.full(mat_in.shape, True)
    if rng is None:
        rng = np.random.default_rng()
    
    mat_out = mat_in.copy()
    mat_out[mask] = rng.choice(mat_in[mask],size=mat_in[mask].size, replace=replace)
    return mat_out

def mix_up_columns_under_mask(mat_in,mask=None,rng=None,permute = True):
    if mask is None:
        mask = np.full(mat_in.shape, True)
    if rng is None:
        rng = np.random.default_rng()
    
    mat_out = mat_in.copy()
    for r in range(mask.shape[0]):
        if mask[r,0]:
            if permute:
                mat_out[r,:] = mat_out[r,rng.permutation(mask.shape[1])]
            else:
                mat_out[r,:] = mat_out[r,rng.integers(0,high=mask.shape[1],size=5)]
            
    return mat_out



# Main Tibia Flexor

## Starting point

In [ ]:
# The starting point

All = slice(None)
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
sim_mat = cosine_similarity(femur_df_local.to_numpy().transpose())
plot_sim_map(sim_mat,femur_df_local)
plot_conn_map(femur_df_local.to_numpy(),femur_df_local)

## Mean of pairwise similarity, shuffling weights within hemilineage = Density, No propotionality


In [ ]:
All = slice(None)
mtf_local_df = pre_to_mn_df.loc[('local',All,All,All,All),mpool_dict['main_tibia_flexor_muscle']]
mtf_local_df = pre_to_mn_df.loc[('local','main_tibia_flexor',All,All,All),mpool_dict['main_tibia_flexor_muscle']]
sim_mat = cosine_similarity(mtf_local_df.to_numpy().transpose())

pair_wise_comparisons = np.tril(np.full(sim_mat.shape,True),k=-1)
cos_sim_mean = sim_mat[pair_wise_comparisons].mean()
print(cos_sim_mean) # note, not that different if just using the preferred pool, so just limit to the small square

plot_sim_map(sim_mat,mtf_local_df)
plot_conn_map(mtf_local_df.to_numpy(),mtf_local_df,fn='MTF_full_conn')

## Get the top celltypes

In [ ]:
mtf_local_df.index.get_level_values('NT').unique()
mtf_local_df.index.get_level_values('classification_system').unique()
cell_types = mtf_local_df.index.get_level_values('cell_type').value_counts().index.to_list()
cell_types = cell_types[0:6]
cell_types
mtf_local_df.index.get_level_values('cell_type').value_counts()

## Make some masks

In [ ]:
mask_df = pd.DataFrame(data=np.full(mtf_local_df.shape, False),index=mtf_local_df.index,columns=mtf_local_df.columns)

# allocate an empty array
N=10000

mixups = np.zeros((N,sim_mat[pair_wise_comparisons].size))

figname = 'mix_all'

# mix up all preMNs with preferred pool
mask_df.loc[:,:] = False
mask_df.loc[('local','main_tibia_flexor',All,All,All,All),:] = True
mask_pool = mask_df.to_numpy().copy()
figname = 'mix_all'


mask_list = []
# mix up hemilineage preMNs with preferred pool
for ct in cell_types:
    mask_df.loc[:,:] = False
    mask_df.loc[(All,'main_tibia_flexor',All,All,ct,All),:] = True
    mask_list.append(mask_df.to_numpy().copy())

mixups = np.zeros((N,sim_mat[pair_wise_comparisons].size))
# figname = 'mix_hl_{}'.format(cell_type)

# Check that the masks are correct
if True: # Check the masks
    for ct,mask in zip(cell_types,mask_list):
        plot_mask(mask,mtf_local_df,fn='MTF_mask_{}'.format(ct))



rng = np.random.default_rng()
mixupmat = mtf_local_df.to_numpy().copy()

pltflp=True
for r in range(N): # range(N):
    # Do this for all of the masks
    mixup = choice_under_mask(mixupmat,mask=mask_pool,rng=rng)
    # mixup = mix_up_under_mask(mixupmat,mask=mask_pool,rng=rng)
    sim_mat = cosine_similarity(mixup.transpose())
    mixups[r,:] = sim_mat[pair_wise_comparisons]


    if r>9998 and pltflp: # Check the mixups
        plot_conn_map(mixup,mtf_local_df)
        pltflp=False

mixup_label = ['pool']
mixups_list = [mixups]
plot_shuf_cum(mixups.mean(axis=1),ref_x=cos_sim_mean)

## Shuffle, with replacement, within hemilineages

In [ ]:
rng = np.random.default_rng()
mixupmat = mtf_local_df.to_numpy().copy()
cnt = 0

for ct,mask in zip(cell_types,mask_list):
    pltflp=False
    for r in range(N): # range(N):
        # Do this for all of the masks
        #mixup = choice_under_mask(mixupmat,mask=mask,rng=rng)
        mixup = mix_up_under_mask(mixupmat,mask=mask,rng=rng)
        sim_mat = cosine_similarity(mixup.transpose())
        mixups[r,:] = sim_mat[pair_wise_comparisons]


        if r>9998 and pltflp: # Check the mixups
            print(mtf_local_df.loc[('local','main_tibia_flexor',All,All,ct,All)].index.get_level_values('segID').to_list())
            plot_hem_conn_map(mixup,mtf_local_df,ct,fn='MTF_shuffle_{}'.format(ct))
            pltflp=False

    mixup_label.append(ct)
    mixups_list.append(mixups.copy())
    # plot_shuf_cum(mixups.mean(axis=1),ref_x=cos_sim_mean,fignum=cnt,title='Hemilineage: {}'.format(ct),fn='MTF_bootstrap_cum_{}'.format(ct))
    # plt.show()
    cnt = cnt+1


In [ ]:
fig = plt.figure(1, figsize=(10, 6))
ax = plt.subplot2grid((1,1),(0,0))
ax.set_ylim([-.1, 1.1])
ax.set_xlim([.945, .96])
ax.set_xlim([0, 1])
ax.set_ylabel('Cummulative fraction')
ax.set_xlabel('Sum of cosine similarity for MTF')
ax.set_title('Shuffling different groups')

# if not fn is None:
#     fig.savefig('./figpanels/{}.svg'.format(fn),format='svg')

colors = sns.color_palette("husl", len(mixup_label))

for ct,mixups,clr in zip(mixup_label,mixups_list,colors):
    # colors = {1: '#CCCCCC', 2: '#666666',3: '#000000'}
    # for m in [1,2,3]:
    # exec('mu=mixups_{}'.format(m))
    x_unsorted = mixups.mean(axis=1)
    y = np.array(range(len(x_unsorted))).astype(float)/len(x_unsorted)

    pct_below_actual = (x_unsorted<cos_sim_mean).sum() / len(x_unsorted)
    p=1-pct_below_actual

    lbl = '{}, p = {:.5f}'.format(ct,p)
    ax.plot(sorted(x_unsorted),y,color=clr,label=lbl)

ax.plot(cos_sim_mean*np.array([1,1]),np.array([0,1]),color='#0000FF')
ax.legend()

fig.savefig('./figpanels/MTF_shuffle_cum_all.svg',format='svg')

## Do the same thing, but just shuffle columns

In [ ]:
mtf_local_df.index.get_level_values('NT').unique()
mtf_local_df.index.get_level_values('classification_system').unique()
cell_types = mtf_local_df.index.get_level_values('cell_type').value_counts().index.to_list()
cell_types = cell_types[0:6]
cell_types
mtf_local_df.index.get_level_values('cell_type').value_counts()

mask_df = pd.DataFrame(data=np.full(mtf_local_df.shape, False),index=mtf_local_df.index,columns=mtf_local_df.columns)

# allocate an empty array
N=10000

mixups = np.zeros((N,sim_mat[pair_wise_comparisons].size))

figname = 'mix_all'

# mix up all preMNs with preferred pool
mask_df.loc[:,:] = False
mask_df.loc[('local','main_tibia_flexor',All,All,All,All),:] = True
mask_pool = mask_df.to_numpy().copy()
figname = 'mix_all'


mask_list = []
# mix up hemilineage preMNs with preferred pool
for ct in cell_types:
    mask_df.loc[:,:] = False
    mask_df.loc[(All,'main_tibia_flexor',All,All,ct,All),:] = True
    mask_list.append(mask_df.to_numpy().copy())

mixups = np.zeros((N,sim_mat[pair_wise_comparisons].size))
# figname = 'mix_hl_{}'.format(cell_type)

# Check that the masks are correct
if False: # Check the masks
    for ct,mask in zip(cell_types,mask_list):
        plot_mask(mask,mtf_local_df,) # fn='MTF_mask_{}'.format(ct)



rng = np.random.default_rng()
mixupmat = mtf_local_df.to_numpy().copy()

pltflp=True
for r in range(N): # range(N):
    # Do this for all of the masks
    # mixup = choice_under_mask(mixupmat,mask=mask_pool,rng=rng)
    # mixup = mix_up_under_mask(mixupmat,mask=mask_pool,rng=rng)
    mixup = mix_up_columns_under_mask(mixupmat,mask=mask_pool,rng=rng)
    sim_mat = cosine_similarity(mixup.transpose())
    mixups[r,:] = sim_mat[pair_wise_comparisons]


    if r>9998 and pltflp: # Check the mixups
        plot_conn_map(mixup,mtf_local_df,'MTF_dense_not_proportional')
        pltflp=False

mixup_label = ['pool']
mixups_list = [mixups.copy()]
plot_shuf_cum(mixups.mean(axis=1),ref_x=cos_sim_mean,fn='MTF_dense_not_proportional_shufcum')

## Shuffle, with replacement, within hemilineages

In [ ]:
rng = np.random.default_rng()
mixupmat = mtf_local_df.to_numpy().copy()
cnt = 0

for ct,mask in zip(cell_types,mask_list):
    pltflp=False
    for r in range(N): # range(N):
        # Do this for all of the masks
        #mixup = choice_under_mask(mixupmat,mask=mask,rng=rng)
        # mixup = mix_up_under_mask(mixupmat,mask=mask,rng=rng)
        mixup = mix_up_columns_under_mask(mixupmat,mask=mask_pool,rng=rng)
        sim_mat = cosine_similarity(mixup.transpose())
        mixups[r,:] = sim_mat[pair_wise_comparisons]


        if r>9998 and pltflp: # Check the mixups
            print(mtf_local_df.loc[('local','main_tibia_flexor',All,All,ct,All)].index.get_level_values('segID').to_list())
            plot_hem_conn_map(mixup,mtf_local_df,ct,fn='MTF_shuffle_{}'.format(ct))
            pltflp=False

    mixup_label.append(ct)
    mixups_list.append(mixups.copy())
    # plot_shuf_cum(mixups.mean(axis=1),ref_x=cos_sim_mean,fignum=cnt,title='Hemilineage: {}'.format(ct),fn='MTF_bootstrap_cum_{}'.format(ct))
    # plt.show()
    cnt = cnt+1


In [ ]:
fig = plt.figure(1, figsize=(10, 6))
ax = plt.subplot2grid((1,1),(0,0))
ax.set_ylim([-.1, 1.1])
ax.set_xlim([.945, .96])
ax.set_xlim([0, 1])
ax.set_ylabel('Cummulative fraction')
ax.set_xlabel('Sum of cosine similarity for MTF')
ax.set_title('Shuffling different groups')

# if not fn is None:
#     fig.savefig('./figpanels/{}.svg'.format(fn),format='svg')

colors = sns.color_palette("husl", len(mixup_label))

for ct,mixups,clr in zip(mixup_label,mixups_list,colors):
    # colors = {1: '#CCCCCC', 2: '#666666',3: '#000000'}
    # for m in [1,2,3]:
    # exec('mu=mixups_{}'.format(m))
    x_unsorted = mixups.mean(axis=1)
    y = np.array(range(len(x_unsorted))).astype(float)/len(x_unsorted)

    pct_below_actual = (x_unsorted<cos_sim_mean).sum() / len(x_unsorted)
    p=1-pct_below_actual

    lbl = '{}, p = {:.5f}'.format(ct,p)
    ax.plot(sorted(x_unsorted),y,color=clr,label=lbl)

ax.plot(cos_sim_mean*np.array([1,1]),np.array([0,1]),color='#0000FF')
ax.legend()

fig.savefig('./figpanels/MTF_shuffle_cum_all.svg',format='svg')

# Tibia extensors

## Starting point

In [ ]:
## Starting point
# The starting point

All = slice(None)
femur_df_local = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]
sim_mat = cosine_similarity(femur_df_all.to_numpy().transpose())
plot_sim_map(sim_mat,femur_df_local)
plot_conn_map(femur_df_local.to_numpy(),femur_df_local)


## FETi and SETi connectivity

In [ ]:

All = slice(None)
# SFTi_local_df = pre_to_mn_df.loc[('local',All,All,All,All),mpool_dict['main_tibia_flexor_muscle']]
SFTi_local_df = pre_to_mn_df.loc[('local','tibia_extensor',All,All,All),mpool_dict['tibia_extensor']]
sim_mat = cosine_similarity(SFTi_local_df.to_numpy().transpose())

pair_wise_comparisons = np.tril(np.full(sim_mat.shape,True),k=-1)
cos_sim_mean = sim_mat[pair_wise_comparisons].mean()
print(cos_sim_mean) # note, not that different if just using the preferred pool, so just limit to the small square

plot_sim_map(sim_mat,SFTi_local_df)
plot_conn_map(SFTi_local_df.to_numpy(),SFTi_local_df,fn='SFTi_full_conn')

In [ ]:
SFTi_local_df.index.get_level_values('NT').unique()
SFTi_local_df.index.get_level_values('classification_system').unique()
cell_types = SFTi_local_df.index.get_level_values('cell_type').value_counts().index.to_list()
cell_types = cell_types[0:6]
cell_types
SFTi_local_df.index.get_level_values('cell_type').value_counts()


translate_df = pd.DataFrame.from_dict({'cell_type':['14A','14A_','4B','4B-','4B_'] ,'new':['14A','14A','4B','4B','4B'] })
translate_df.set_index('cell_type',inplace=True)
sft_mi = SFTi_local_df.index.to_frame().reset_index(drop=True)
sft_mi_new = sft_mi.join(translate_df,on='cell_type',how='left',lsuffix='',rsuffix='_r')
sft_mi_new.loc[~sft_mi_new.new.isna(),'cell_type'] = sft_mi_new.loc[~sft_mi_new.new.isna(),'new']
sft_mi_new.cell_type.value_counts()
cell_types = sft_mi_new.cell_type.value_counts().index.to_list()
cell_types = cell_types[0:6]

SFTi_local_df.index = pd.MultiIndex.from_frame(sft_mi_new)

In [ ]:
sft_mi_new.cell_type.value_counts()

## Shuffle, without replacement, for the pool and hemilineages

In [ ]:
mask_df = pd.DataFrame(data=np.full(SFTi_local_df.shape, False),index=SFTi_local_df.index,columns=SFTi_local_df.columns)

# allocate an empty array
N=10000

mixups = np.zeros((N,sim_mat[pair_wise_comparisons].size))

figname = 'mix_all'

# mix up all preMNs with preferred pool
mask_df.loc[:,:] = False
mask_df.loc[('local','tibia_extensor',All,All,All,All),:] = True
mask_pool = mask_df.to_numpy().copy()
figname = 'mix_all'


mask_list = []
# mix up hemilineage preMNs with preferred pool
for ct in cell_types:
    mask_df.loc[:,:] = False
    mask_df.loc[(All,'tibia_extensor',All,All,ct,All),:] = True
    mask_list.append(mask_df.to_numpy().copy())

mixups = np.zeros((N,sim_mat[pair_wise_comparisons].size))
# figname = 'mix_hl_{}'.format(cell_type)

# Check that the masks are correct
if False: # Check the masks
    for ct,mask in zip(cell_types,mask_list):
        plot_mask(mask,SFTi_local_df,fn='MTF_mask_{}'.format(ct))


rng = np.random.default_rng()
mixupmat = SFTi_local_df.to_numpy().copy()

pltflp=False
for r in range(N): # range(N):
    # Do this for all of the masks
    mixup = choice_under_mask(mixupmat,mask=mask_pool,rng=rng)
    # mixup = mix_up_under_mask(mixupmat,mask=mask_pool,rng=rng)
    sim_mat = cosine_similarity(mixup.transpose())
    mixups[r,:] = sim_mat[pair_wise_comparisons]


    if r>9998 and pltflp: # Check the mixups
        plot_conn_map(mixup,SFTi_local_df)
        pltflp=False

mixup_label = ['pool']
mixups_list = [mixups.copy()]
# plot_shuf_cum(mixups.mean(axis=1),ref_x=cos_sim_mean)

rng = np.random.default_rng()
mixupmat = SFTi_local_df.to_numpy().copy()
cnt = 0

for ct,mask in zip(cell_types,mask_list):
    pltflp=False
    for r in range(N): # range(N):
        # Do this for all of the masks
        #mixup = choice_under_mask(mixupmat,mask=mask,rng=rng)
        mixup = mix_up_under_mask(mixupmat,mask=mask,rng=rng)
        sim_mat = cosine_similarity(mixup.transpose())
        mixups[r,:] = sim_mat[pair_wise_comparisons]


        if r>9998 and pltflp: # Check the mixups
            print(SFTi_local_df.loc[('local','main_tibia_flexor',All,All,ct,All)].index.get_level_values('segID').to_list())
            plot_hem_conn_map(mixup,SFTi_local_df,ct,fn='MTF_shuffle_{}'.format(ct))
            pltflp=False

    mixup_label.append(ct)
    mixups_list.append(mixups.copy())
    # plot_shuf_cum(mixups.mean(axis=1),ref_x=cos_sim_mean,fignum=cnt,title='Hemilineage: {}'.format(ct),fn='MTF_bootstrap_cum_{}'.format(ct))
    # plt.show()
    cnt = cnt+1

fig = plt.figure(1, figsize=(10, 6))
ax = plt.subplot2grid((1,1),(0,0))
ax.set_ylim([-.1, 1.1])
ax.set_xlim([.945, .96])
ax.set_xlim([0, 1])
ax.set_ylabel('Cummulative fraction')
ax.set_xlabel('Sum of cosine similarity for MTF')
ax.set_title('Shuffling different groups')

# if not fn is None:
#     fig.savefig('./figpanels/{}.svg'.format(fn),format='svg')

colors = sns.color_palette("husl", len(mixup_label))

for ct,mixups,clr in zip(mixup_label,mixups_list,colors):
    # colors = {1: '#CCCCCC', 2: '#666666',3: '#000000'}
    # for m in [1,2,3]:
    # exec('mu=mixups_{}'.format(m))
    x_unsorted = mixups.mean(axis=1)
    y = np.array(range(len(x_unsorted))).astype(float)/len(x_unsorted)

    pct_below_actual = (x_unsorted<cos_sim_mean).sum() / len(x_unsorted)
    p=1-pct_below_actual

    lbl = '{}, p = {:.5f}'.format(ct,p)
    ax.plot(sorted(x_unsorted),y,color=clr,label=lbl)

ax.plot(cos_sim_mean*np.array([1,1]),np.array([0,1]),color='#0000FF')
ax.legend()

fig.savefig('./figpanels/STF_shuffle_cum_all.svg',format='svg')

# Femur flexors

## Starting point

In [ ]:
## Starting point
# The starting point

All = slice(None)
coxa_df_local = pre_to_mn_df.loc[('local'),('L',All,'coxa',All,All,['Ac','Vn'],All)]
sim_mat = cosine_similarity(coxa_df_local.to_numpy().transpose())
plot_sim_map(sim_mat,coxa_df_local)
plot_conn_map(coxa_df_local.to_numpy(),coxa_df_local)


In [ ]:

All = slice(None)
# SFTi_local_df = pre_to_mn_df.loc[('local',All,All,All,All),mpool_dict['main_tibia_flexor_muscle']]
# SFTi_local_df = pre_to_mn_df.loc[('local','tibia_extensor',All,All,All),mpool_dict['tibia_extensor']]
coxa_df_local = pre_to_mn_df.loc[('local','trochanter_flexion',All,All,All),('L',All,'coxa',All,All,['Ac','Vn'],All)]
sim_mat = cosine_similarity(coxa_df_local.to_numpy().transpose())

pair_wise_comparisons = np.tril(np.full(sim_mat.shape,True),k=-1)
cos_sim_mean = sim_mat[pair_wise_comparisons].mean()
print(cos_sim_mean) # note, not that different if just using the preferred pool, so just limit to the small square

plot_sim_map(sim_mat,coxa_df_local)
plot_conn_map(coxa_df_local.to_numpy(),coxa_df_local,fn='trflex_full_conn')

In [ ]:
coxa_df_local.index.get_level_values('NT').unique()
coxa_df_local.index.get_level_values('classification_system').unique()

translate_df = pd.DataFrame.from_dict({'cell_type':['14A','14A_','4B','4B-','4B_','22A_','8A_','21A_'] ,'new':['14A','14A','4B','4B','4B','22A','8A','21A'] })
translate_df.set_index('cell_type',inplace=True)
cf_mi = coxa_df_local.index.to_frame().reset_index(drop=True)
cf_mi_new = cf_mi.join(translate_df,on='cell_type',how='left',lsuffix='',rsuffix='_r')
cf_mi_new.loc[~cf_mi_new.new.isna(),'cell_type'] = cf_mi_new.loc[~cf_mi_new.new.isna(),'new']
cf_mi_new.cell_type.value_counts()
cell_types = cf_mi_new.cell_type.value_counts().index.to_list()
cell_types = cell_types[0:6]

coxa_df_local.index = pd.MultiIndex.from_frame(cf_mi_new)
cell_types

In [ ]:
coxa_df_local.index.get_level_values('cell_type').value_counts()

## Shuffle, without replacement, for the pool and hemilineages

In [ ]:
mask_df = pd.DataFrame(data=np.full(coxa_df_local.shape, False),index=coxa_df_local.index,columns=coxa_df_local.columns)

N=10000

# mix up all preMNs with preferred pool
mask_df.loc[:,:] = False
mask_df.loc[('local','trochanter_flexion',All,All,All,All),:] = True
mask_pool = mask_df.to_numpy().copy()

mask_list = []
# mix up hemilineage preMNs with preferred pool
for ct in cell_types:
    mask_df.loc[:,:] = False
    mask_df.loc[(All,'trochanter_flexion',All,All,ct,All),:] = True
    mask_list.append(mask_df.to_numpy().copy())

mixups = np.zeros((N,sim_mat[pair_wise_comparisons].size))
# figname = 'mix_hl_{}'.format(cell_type)

# Check that the masks are correct
if True: # Check the masks
    for ct,mask in zip(cell_types,mask_list):
        plot_mask(mask,coxa_df_local,fn='trflex_mask_{}'.format(ct))


rng = np.random.default_rng()
mixupmat = coxa_df_local.to_numpy().copy()

pltflp=False
for r in range(N): # range(N):
    # Do this for all of the masks
    mixup = choice_under_mask(mixupmat,mask=mask_pool,rng=rng)
    # mixup = mix_up_under_mask(mixupmat,mask=mask_pool,rng=rng)
    sim_mat = cosine_similarity(mixup.transpose())
    mixups[r,:] = sim_mat[pair_wise_comparisons]


    if r>9998 and pltflp: # Check the mixups
        plot_conn_map(mixup,coxa_df_local)
        pltflp=False

mixup_label = ['pool']
mixups_list = [mixups.copy()]
# plot_shuf_cum(mixups.mean(axis=1),ref_x=cos_sim_mean)

rng = np.random.default_rng()
mixupmat = coxa_df_local.to_numpy().copy()
cnt = 0

for ct,mask in zip(cell_types,mask_list):
    pltflp=False
    for r in range(N): # range(N):
        # Do this for all of the masks
        #mixup = choice_under_mask(mixupmat,mask=mask,rng=rng)
        mixup = mix_up_under_mask(mixupmat,mask=mask,rng=rng)
        sim_mat = cosine_similarity(mixup.transpose())
        mixups[r,:] = sim_mat[pair_wise_comparisons]


        if r>9998 and pltflp: # Check the mixups
            print(coxa_df_local.loc[('local','main_tibia_flexor',All,All,ct,All)].index.get_level_values('segID').to_list())
            plot_hem_conn_map(mixup,coxa_df_local,ct,fn='MTF_shuffle_{}'.format(ct))
            pltflp=False

    mixup_label.append(ct)
    mixups_list.append(mixups.copy())
    # plot_shuf_cum(mixups.mean(axis=1),ref_x=cos_sim_mean,fignum=cnt,title='Hemilineage: {}'.format(ct),fn='MTF_bootstrap_cum_{}'.format(ct))
    # plt.show()
    cnt = cnt+1

fig = plt.figure(1, figsize=(10, 6))
ax = plt.subplot2grid((1,1),(0,0))
ax.set_ylim([-.1, 1.1])
ax.set_xlim([.945, .96])
ax.set_xlim([0, 1])
ax.set_ylabel('Cummulative fraction')
ax.set_xlabel('Sum of cosine similarity for MTF')
ax.set_title('Shuffling different groups')

# if not fn is None:
#     fig.savefig('./figpanels/{}.svg'.format(fn),format='svg')

colors = sns.color_palette("husl", len(mixup_label))

for ct,mixups,clr in zip(mixup_label,mixups_list,colors):
    # colors = {1: '#CCCCCC', 2: '#666666',3: '#000000'}
    # for m in [1,2,3]:
    # exec('mu=mixups_{}'.format(m))
    x_unsorted = mixups.mean(axis=1)
    y = np.array(range(len(x_unsorted))).astype(float)/len(x_unsorted)

    pct_below_actual = (x_unsorted<cos_sim_mean).sum() / len(x_unsorted)
    p=1-pct_below_actual

    lbl = '{}, p = {:.5f}'.format(ct,p)
    ax.plot(sorted(x_unsorted),y,color=clr,label=lbl)

ax.plot(cos_sim_mean*np.array([1,1]),np.array([0,1]),color='#0000FF')
ax.legend()

fig.savefig('./figpanels/TrFlex_shuffle_cum_all.svg',format='svg')

# Tergotr tr extensors

## Starting point

In [ ]:
pre_to_mn_df.loc[('local','trochanter_extension'),mpool_dict['trochanter_extension']]

In [ ]:
## Starting point
# The starting point

All = slice(None)
tr_ext_local_df = pre_to_mn_df.loc[('local'),('L',All,'thorax',All,'tergotrochanter',All,All)]
sim_mat = cosine_similarity(tr_ext_local_df.to_numpy().transpose())
plot_sim_map(sim_mat,tr_ext_local_df)
plot_conn_map(tr_ext_local_df.to_numpy(),tr_ext_local_df)


In [ ]:

All = slice(None)
# SFTi_local_df = pre_to_mn_df.loc[('local',All,All,All,All),mpool_dict['main_tibia_flexor_muscle']]
# SFTi_local_df = pre_to_mn_df.loc[('local','tibia_extensor',All,All,All),mpool_dict['tibia_extensor']]
# coxa_df_local = pre_to_mn_df.loc[('local','trochanter_flexion',All,All,All),('L',All,'coxa',All,All,['Ac','Vn'],All)]
tr_ext_local_df = pre_to_mn_df.loc[('local','trochanter_extension',All,All,All),('L',All,'thorax',All,'tergotrochanter',All,All)]
sim_mat = cosine_similarity(tr_ext_local_df.to_numpy().transpose())

pair_wise_comparisons = np.tril(np.full(sim_mat.shape,True),k=-1)
cos_sim_mean = sim_mat[pair_wise_comparisons].mean()
print(cos_sim_mean) # note, not that different if just using the preferred pool, so just limit to the small square

plot_sim_map(sim_mat,tr_ext_local_df)
plot_conn_map(tr_ext_local_df.to_numpy(),tr_ext_local_df,fn='trExt_full_conn')

In [ ]:
tr_ext_local_df.index.get_level_values('NT').unique()
tr_ext_local_df.index.get_level_values('classification_system').unique()
tr_ext_local_df.index.get_level_values('cell_type').unique().to_list()

In [ ]:

translate_df = pd.DataFrame.from_dict({'cell_type':['14A','14A_','4B','4B-','4B_','4B+','22A_','8A_','21A_'] ,'new':['14A','14A','4B','4B','4B','4B','22A','8A','21A'] })
translate_df.set_index('cell_type',inplace=True)
tre_mi = tr_ext_local_df.index.to_frame().reset_index(drop=True)
tre_mi_new = tre_mi.join(translate_df,on='cell_type',how='left',lsuffix='',rsuffix='_r')
tre_mi_new.loc[~tre_mi_new.new.isna(),'cell_type'] = tre_mi_new.loc[~tre_mi_new.new.isna(),'new']
tre_mi_new.cell_type.value_counts()
cell_types = tre_mi_new.cell_type.value_counts().index.to_list()
cell_types = cell_types[0:6]

tr_ext_local_df.index = pd.MultiIndex.from_frame(tre_mi_new)
cell_types

In [ ]:
tr_ext_local_df.index.get_level_values('cell_type').value_counts()

## Shuffle, without replacement, for the pool and hemilineages

In [ ]:
mask_df = pd.DataFrame(data=np.full(tr_ext_local_df.shape, False),index=tr_ext_local_df.index,columns=tr_ext_local_df.columns)

N=10000

# mix up all preMNs with preferred pool
mask_df.loc[:,:] = False
mask_df.loc[('local','trochanter_extension',All,All,All,All),:] = True
mask_pool = mask_df.to_numpy().copy()

mask_list = []
# mix up hemilineage preMNs with preferred pool
for ct in cell_types:
    mask_df.loc[:,:] = False
    mask_df.loc[(All,'trochanter_extension',All,All,ct,All),:] = True
    mask_list.append(mask_df.to_numpy().copy())

mixups = np.zeros((N,sim_mat[pair_wise_comparisons].size))
# figname = 'mix_hl_{}'.format(cell_type)

# Check that the masks are correct
if True: # Check the masks
    for ct,mask in zip(cell_types,mask_list):
        plot_mask(mask,tr_ext_local_df,fn='trflex_mask_{}'.format(ct))


rng = np.random.default_rng()
mixupmat = tr_ext_local_df.to_numpy().copy()

pltflp=False
for r in range(N): # range(N):
    # Do this for all of the masks
    mixup = choice_under_mask(mixupmat,mask=mask_pool,rng=rng)
    # mixup = mix_up_under_mask(mixupmat,mask=mask_pool,rng=rng)
    sim_mat = cosine_similarity(mixup.transpose())
    mixups[r,:] = sim_mat[pair_wise_comparisons]


    if r>9998 and pltflp: # Check the mixups
        plot_conn_map(mixup,tr_ext_local_df)
        pltflp=False

mixup_label = ['pool']
mixups_list = [mixups.copy()]
# plot_shuf_cum(mixups.mean(axis=1),ref_x=cos_sim_mean)

rng = np.random.default_rng()
mixupmat = tr_ext_local_df.to_numpy().copy()
cnt = 0

for ct,mask in zip(cell_types,mask_list):
    pltflp=False
    for r in range(N): # range(N):
        # Do this for all of the masks
        #mixup = choice_under_mask(mixupmat,mask=mask,rng=rng)
        mixup = mix_up_under_mask(mixupmat,mask=mask,rng=rng)
        sim_mat = cosine_similarity(mixup.transpose())
        mixups[r,:] = sim_mat[pair_wise_comparisons]


        if r>9998 and pltflp: # Check the mixups
            print(tr_ext_local_df.loc[('local','main_tibia_flexor',All,All,ct,All)].index.get_level_values('segID').to_list())
            plot_hem_conn_map(mixup,tr_ext_local_df,ct,fn='MTF_shuffle_{}'.format(ct))
            pltflp=False

    mixup_label.append(ct)
    mixups_list.append(mixups.copy())
    # plot_shuf_cum(mixups.mean(axis=1),ref_x=cos_sim_mean,fignum=cnt,title='Hemilineage: {}'.format(ct),fn='MTF_bootstrap_cum_{}'.format(ct))
    # plt.show()
    cnt = cnt+1

fig = plt.figure(1, figsize=(10, 6))
ax = plt.subplot2grid((1,1),(0,0))
ax.set_ylim([-.1, 1.1])
ax.set_xlim([.945, .96])
ax.set_xlim([0, 1])
ax.set_ylabel('Cummulative fraction')
ax.set_xlabel('Sum of cosine similarity for MTF')
ax.set_title('Shuffling different groups')

# if not fn is None:
#     fig.savefig('./figpanels/{}.svg'.format(fn),format='svg')

colors = sns.color_palette("husl", len(mixup_label))

for ct,mixups,clr in zip(mixup_label,mixups_list,colors):
    # colors = {1: '#CCCCCC', 2: '#666666',3: '#000000'}
    # for m in [1,2,3]:
    # exec('mu=mixups_{}'.format(m))
    x_unsorted = mixups.mean(axis=1)
    y = np.array(range(len(x_unsorted))).astype(float)/len(x_unsorted)

    pct_below_actual = (x_unsorted<cos_sim_mean).sum() / len(x_unsorted)
    p=1-pct_below_actual

    lbl = '{}, p = {:.5f}'.format(ct,p)
    ax.plot(sorted(x_unsorted),y,color=clr,label=lbl)

ax.plot(cos_sim_mean*np.array([1,1]),np.array([0,1]),color='#0000FF')
ax.legend()

fig.savefig('./figpanels/TrExt_shuffle_cum_all.svg',format='svg')